### Cào urls của tất cả các jobs

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from concurrent.futures import ThreadPoolExecutor, as_completed
import time

# Cấu hình Selenium và đường dẫn ChromeDriver
CHROMEDRIVER_PATH = "D:\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"  # Thay bằng đường dẫn ChromeDriver (Windows)


# Hàm lấy liên kết công việc từ một trang
def fetch_job_links_from_page_selenium(base_url, page):
    url = f"{base_url}tat-ca-viec-lam-trang-{page}-vi.html"
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Chạy không giao diện
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    # Sử dụng Service với ChromeDriver có sẵn
    service = Service(CHROMEDRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.get(url)
    time.sleep(5)  # Đợi cho trang tải xong

    job_links_list = []
    job_links = driver.find_elements(By.CSS_SELECTOR, 'h2 > .job_link')  # Sửa lại CSS selector cho đúng
    job_links_list = [job.get_attribute('href') for job in job_links]

    driver.quit()
    print(f"Lấy được {len(job_links_list)} liên kết công việc từ trang {page}")
    return job_links_list

# Sử dụng Selenium để cào tất cả các trang với đa luồng
def get_all_job_links_selenium_multithread(base_url, start_page=1, max_pages=10, max_workers=5):
    job_links = []
    pages = range(start_page, max_pages + 1)

    # Sử dụng ThreadPoolExecutor để cào đa luồng
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_page = {executor.submit(fetch_job_links_from_page_selenium, base_url, page): page for page in pages}
        for future in as_completed(future_to_page):
            page = future_to_page[future]
            try:
                result = future.result()
                if result:
                    job_links.extend(result)
            except Exception as e:
                print(f"Lỗi khi lấy liên kết từ trang {page}: {e}")

    return job_links

# Sử dụng ví dụ
if __name__ == "__main__":
    BASE_URL = "https://careerviet.vn/viec-lam/"
    job_links = get_all_job_links_selenium_multithread(BASE_URL, max_pages=505, max_workers=10)

    # Lưu liên kết vào tệp
    with open('job_links.txt', 'w', encoding='utf-8') as f:
        for link in job_links:
            f.write(link + '\n')


Lấy được 50 liên kết công việc từ trang 6
Lấy được 50 liên kết công việc từ trang 1
Lấy được 50 liên kết công việc từ trang 2
Lấy được 50 liên kết công việc từ trang 7
Lấy được 50 liên kết công việc từ trang 9
Lấy được 50 liên kết công việc từ trang 4
Lấy được 50 liên kết công việc từ trang 10
Lấy được 50 liên kết công việc từ trang 8
Lấy được 50 liên kết công việc từ trang 3
Lấy được 50 liên kết công việc từ trang 5
Lấy được 50 liên kết công việc từ trang 11
Lấy được 0 liên kết công việc từ trang 15
Lấy được 50 liên kết công việc từ trang 19
Lấy được 50 liên kết công việc từ trang 16
Lấy được 50 liên kết công việc từ trang 12
Lấy được 50 liên kết công việc từ trang 14
Lấy được 50 liên kết công việc từ trang 13
Lấy được 50 liên kết công việc từ trang 20
Lấy được 50 liên kết công việc từ trang 21
Lấy được 50 liên kết công việc từ trang 18
Lấy được 50 liên kết công việc từ trang 22
Lấy được 50 liên kết công việc từ trang 17
Lấy được 50 liên kết công việc từ trang 32
Lấy được 50 liên kết 

### Cào jobs từ URL trong file job_links

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

# Hàm để cào thông tin từ URL công ty (nếu có)
def fetch_company_details(company_url, headers):
    try:
        response = requests.get(company_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Lấy tên công ty
        company_name = soup.select_one("h1.name").text.strip() if soup.select_one("h1.name") else "Not specified"

        # Lấy địa điểm công ty
        company_address = (
            soup.select_one("strong:contains('Địa điểm') + p").text.strip()
            if soup.select_one("strong:contains('Địa điểm') + p")
            else "Not specified"
        )

        # Lấy quy mô công ty (loại bỏ phần đầu "Quy mô công ty:")
        company_size_element = soup.select_one("li:has(.mdi-account-supervisor)")
        company_size = (
            company_size_element.text.replace("Qui mô công ty:", "").strip()
            if company_size_element else "Not specified"
        )

        # Lấy loại hình hoạt động (loại bỏ phần đầu "Loại hình hoạt động:")
        company_type_element = soup.select_one("li:has(.mdi-gavel)")
        company_type = (
            company_type_element.text.replace("Loại hình hoạt động:", "").strip()
            if company_type_element else "Not specified"
        )

        # Lấy đường dẫn website công ty
        company_website_element = soup.select_one("li:has(.mdi-link)")
        company_website = company_website_element.text.replace("Website: ", "").strip() if company_website_element else "Not specified"

        # Lấy số lượng người theo dõi
        follow_element = soup.select_one("#countFollowers")
        follower = follow_element.text if follow_element else "Not Found"

        # Lấy giới thiệu công ty
        company_intro_element = soup.select_one(".intro-section .main-text")
        company_intro = company_intro_element.text.strip() if company_intro_element else "Not specified"

        return {
            "Company Name": company_name,
            "Company Address": company_address,
            "Company Scale": company_size,
            "Company Type": company_type,
            "Company Website": company_website,
            "Follower": follower,
            "Company Introduction": company_intro
        }
    except requests.RequestException as e:
        print(f"Error fetching company details from '{company_url}': {e}")
        return {
            "Company Name": "Not specified",
            "Company Address": "Not specified",
            "Company Scale": "Not specified",
            "Company Type": "Not specified",
            "Company Website": "Not specified",
            "Follower": "Not specified",
            "Company Introduction": "Not specified"
        }

# Hàm cào dữ liệu từ một URL công việc
def crawl_single_job(job_url, headers):
    try:
        print(f"Crawling: {job_url}")
        response = requests.get(job_url, headers=headers, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')
        time.sleep(random.uniform(1, 3))  # Chờ ngẫu nhiên để tránh bị chặn

        # Lấy tên công việc
        job_name = soup.select_one('h1.title').text.strip() if soup.select_one('h1.title') else "Not specified"

        # Lấy thông tin lương
        salary_info = (
            soup.select_one("strong:contains('Lương') + p").text.strip()
            if soup.select_one("strong:contains('Lương') + p")
            else "Not specified"
        )

        # Lấy địa điểm
        location_elements = soup.select('.map p a')
        locations = ', '.join([loc.text.strip() for loc in location_elements]) if location_elements else "Not specified"

        # Lấy các thông tin khác
        experience = (
            soup.select_one("strong:contains('Kinh nghiệm') + p").text.strip()
            if soup.select_one("strong:contains('Kinh nghiệm') + p")
            else "Not specified"
        )
        job_level = (
            soup.select_one("strong:contains('Cấp bậc') + p").text.strip()
            if soup.select_one("strong:contains('Cấp bậc') + p")
            else "Not specified"
        )
        deadline = (
            soup.select_one("strong:contains('Hết hạn nộp') + p").text.strip()
            if soup.select_one("strong:contains('Hết hạn nộp') + p")
            else "Not specified"
        )
        update_date = (
            soup.select_one("strong:contains('Ngày cập nhật') + p").text.strip()
            if soup.select_one("strong:contains('Ngày cập nhật') + p")
            else "Not specified"
        )
        employment_type = (
            soup.select_one("strong:contains('Hình thức') + p").text.strip()
            if soup.select_one("strong:contains('Hình thức') + p")
            else "Not specified"
        )

        # Lấy loại ngành nghề
        briefcase_element = soup.select_one("li:has(.mdi-briefcase)")
        list_briefcase = ", ".join([a.text for a in briefcase_element.find_all("a")]) if briefcase_element else "Not Found"

        # Lấy danh sách phúc lợi
        welfare_elements = soup.select("ul.welfare-list li")
        welfare_list = ', '.join([welfare.text.strip() for welfare in welfare_elements]) if welfare_elements else "Not specified"

        # Lấy phần "Mô tả Công việc"
        job_description_section = soup.select("h2:contains('Mô tả Công việc') ~ *")
        job_description_list = []
        for section in job_description_section:
            if section.name == "p" and section.text.strip():
                job_description_list.append(section.text.strip())
            elif section.name == "ul":
                description_items = section.find_all("li")
                job_description_list.extend([item.text.strip() for item in description_items if item.text.strip()])
        job_description = '\n'.join(job_description_list) if job_description_list else "Not specified"

        # Lấy phần "Yêu Cầu Công Việc"
        job_requirements_section = soup.select("h2:contains('Yêu Cầu Công Việc') ~ *")
        job_requirements_list = []
        for section in job_requirements_section:
            if section.name == "p" and section.text.strip():
                job_requirements_list.append(section.text.strip())
            elif section.name == "ul":
                requirements_items = section.find_all("li")
                job_requirements_list.extend([item.text.strip() for item in requirements_items if item.text.strip()])
        job_requirements = '\n'.join(job_requirements_list) if job_requirements_list else "Not specified"

        # Lấy nội dung của phần "Thông tin khác"
        other_info_section = soup.select_one("h3:contains('Thông tin khác') + div ul")
        other_info = '\n'.join([item.text.strip() for item in other_info_section.find_all("li")]) if other_info_section else "Not specified"

        # Lấy danh sách các Job Tags / Skills
        job_tags_elements = soup.select("div.job-tags ul li")
        job_tags = ', '.join([tag.text.strip() for tag in job_tags_elements]) if job_tags_elements else "Not specified"

        # Lấy URL công ty từ điều hướng
        company_url_element = soup.select_one("#tabs-job-company a")
        company_url = company_url_element['href'] if company_url_element else "Not Found"
        company_details = fetch_company_details(company_url, headers) if company_url != "Not Found" else {}

        # Tạo thông tin công việc
        job_information = {
            "Job Name": job_name,
            "Job Link": job_url,
            "Location": locations,
            "Update Date": update_date,
            "Deadline": deadline,
            "Experience": experience,
            "Job Level": job_level,
            "Industry": list_briefcase,
            "Employment Type": employment_type,
            "Welfare": welfare_list,
            "Job Description": job_description,
            "Job Requirements": job_requirements,
            "Other Information": other_info,
            "Job Tags": job_tags,
            "Company URL": company_url,
            **company_details,  # Thêm chi tiết công ty
            "Salary": salary_info
        }
        return job_information

    except requests.RequestException as e:
        print(f"Error fetching data from '{job_url}': {e}")
        return None

def crawl_job_information(job_urls):
    job_information_list = []
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }
    
    total_urls = len(job_urls)
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(crawl_single_job, url, headers) for url in job_urls]
        for i, future in enumerate(as_completed(futures), 1):
            result = future.result()
            if result:
                job_information_list.append(result)
            # Print progress
            print(f"Crawled {i}/{total_urls} URLs")
    
    return pd.DataFrame(job_information_list)


# Đọc các link từ file txt
with open('job_links.txt', 'r', encoding='utf-8') as f:
    job_links_from_file = [line.strip() for line in f]

# Crawl job information based on collected job links
job_df = crawl_job_information(job_links_from_file)

# Print the DataFrame
job_df

Crawling: https://careerviet.vn/vi/tim-viec-lam/giam-sat-kho.35C274E6.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/ky-thuat-vien-quy-trinh.35C274CF.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/quan-ly-trang-trai-binh-duong.35C274C5.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/truong-phong-dau-tu-va-phat-trien-du-an-bat-dong-san.35C27462.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-ho-tro-thanh-toan-khoan-vay-hien-truong-binh-duong.35C27465.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/designer.35C27447.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-phat-trien-thi-truong-di-an-thuan-an.35C27440.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/chief-accountant.35C273B9.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-ky-thuat-may-o-to.35C27407.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/thu-kho-ben-cat-binh-duong.35C27402.html
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-media.35C273E

d:\SASM\anaconda\Lib\site-packages\soupsieve\css_parser.py:856: FutureWarning: The pseudo class ':contains' is deprecated, ':-soup-contains' should be used moving forward.
  warnings.warn(  # noqa: B028


Crawling: https://careerviet.vn/vi/tim-viec-lam/giam-doc-khach-hang-doanh-nghiep-mm.35C1C33E.html
Crawled 1/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/it-business-analysis-bi-specialist-chuyen-vien-phan-tich-nghiep-vu-cntt-bao-cao-thong-minh.35C1C46D.html
Crawled 2/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/technical-designer-thiet-ke-ky-thuat-nha-may-ben-cat-binh-duong.35C1C298.html
Crawled 3/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/ky-su-co-khi.35C1C440.html
Crawled 4/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/chuyen-gia-quan-ly-quan-he-khach-hang-doanh-nghiep-usme.35C1C33A.html
Crawled 5/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/nhan-vien-phat-trien-kinh-doanh.35C1C3AC.html
Crawled 6/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/it-security.35C1C3A1.html
Crawled 7/23530 URLs
Crawling: https://careerviet.vn/vi/tim-viec-lam/111012-chief-accountant-manufacturing.35C1C2DF.html
Crawled 8/23530

,Job Name,Job Link,Location,Update Date,Deadline,Experience,Job Level,Industry,Employment Type,Welfare,...,Job Tags,Company URL,Salary,Company Name,Company Address,Company Scale,Company Type,Company Website,Follower,Company Introduction
0,Chuyên viên R&D,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,Bình Dương,19/10/2024,17/11/2024,1 - 2 \r\n ...,Nhân viên,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồng phục, ...",...,"R D Staff, Trợ lý nghiên cứu và phát triển, Re...",Not Found,8 Tr - 11 Tr VND,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,QA Staff,https://careerviet.vn/vi/tim-viec-lam/qa-staff...,Bình Dương,18/11/2024,15/12/2024,Trên 1 \r\n ...,Nhân viên,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Chế độ thưởng, Chăm ...",...,"qa staff, nhân viên kcs, nhân viên quản lý chấ...",https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Cạnh tranh,Công Ty Hansoll Vina,"Số 6,KCN Sóng Thần 1,Dĩ An",~5000,100% vốn nước ngoài,www.hansoll.com,298,Công ty Hansoll Vina có vốn đầu tư 100% nước n...
2,Logistics Staff - VSIP 2 Binh Duong,https://careerviet.vn/vi/tim-viec-lam/logistic...,Bình Dương,18/11/2024,19/12/2024,1 - 3 \r\n ...,Nhân viên,\r\n ...,Nhân viên chính thức,"Laptop, Chế độ bảo hiểm, Du Lịch, Chế độ thưởn...",...,"chuyên viên phụ trách xuất nhập khẩu, nhân viê...",https://careerviet.vn/vi/nha-tuyen-dung/nnr-gl...,Trên 600 USD,"NNR GLOBAL LOGISTICS (VIET NAM) Co., LTD","05 Floor, SCSC Building, 30 Phan Thuc Duyen, W...",100-199,Trách nhiệm hữu hạn,http://www.nnrglobal.com/,280,NNR Global Logistics is the subsidiary of Nish...
3,Not specified,https://careerviet.vn/vi/tim-viec-lam/giam-sat...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,"Chế độ bảo hiểm, Phụ cấp, Đồng phục, Chế độ th...",...,Not specified,Not Found,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Nhân viên công nghệ,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Bình Dương,18/11/2024,31/12/2024,Not specified,Nhân viên,\r\n ...,Nhân viên chính thức,"Chế độ bảo hiểm, Du Lịch, Phụ cấp, Đồng phục, ...",...,Not specified,https://careerviet.vn/vi/nha-tuyen-dung/cong-t...,Cạnh tranh,Công ty Cổ Phần Nhựa Thiếu Niên Tiền Phong Phí...,"LÔ C2, KCN ĐỒNG AN 2, P.HÒA PHÚ, TP THỦ DẦU MỘ...",500-999,Cổ phần,http://nhuatienphong.vn/,4.412,NHỰA TIỀN PHONG - ỐNG NHỰA SỐ 1 VIỆT NAMCông t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23490,Not specified,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,Not specified,...,Not specified,Not Found,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23491,Not specified,https://careerviet.vn/vi/tim-viec-lam/chuyen-v...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,Not specified,...,Not specified,Not Found,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23492,Not specified,https://careerviet.vn/vi/tim-viec-lam/product-...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,Not specified,...,Not specified,Not Found,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23493,Not specified,https://careerviet.vn/vi/tim-viec-lam/nhan-vie...,Not specified,Not specified,Not specified,Not specified,Not specified,Not Found,Not specified,Not specified,...,Not specified,Not Found,Not specified,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
job_df.to_csv('job_data.csv', index=False, encoding='utf-8-sig')